# 0. Install Dependencies

In [ ]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
!pip install tensorflow==2.11.0

  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.10.1 requires keras<2.11,>=2.10.0, but you have keras 2.11.0 which is incompatible.
tensorflow-gpu 2.10.1 requires tensorboard<2.11,>=2.10, but you have tensorboard 2.11.0 which is incompatible.
tensorflow-gpu 2.10.1 requires tensorflow-estimator<2.11,>=2.10.0, but you have tensorflow-estimator 2.11.0 which is incompatible.


In [3]:
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
#build 4 functions init, step, render, reset
# __init__ class for all the stuff which should be loaded at start
#step function is running for each step in our environment
#Env is imported from gym and we are inhereting methods and properties from the OpenAI gym environment
#(python class inheritance)


class ShowerEnvCustom(Env): 

    def __init__(self):
        pass
    def step(self):
        pass
    def render(self):
        pass
    def rest(self):
        pass



In [ ]:
#build 4 functions init, step, render, reset
#init for the inital state
#step function is running for each step in our environment

class ShowerEnvCustom(Env): 
#Env is imported from gym and we are inhereting methods and properties from the OpenAI gym environment
#(python class inheritance)

    def __init__(self):
# __init__ class for all the stuff which should be loaded at start

        self.action_space = Discrete(3)
# action_space is a variable name | 3 Actions we can take => down, stay, up |  Discrete(3, start=-1)  # {-1, 0, 1}  
        
    
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
#Temperature array scale, points where the state is https://www.gymlibrary.dev/api/spaces/#gym.spaces.Box 
#the box space gives you a lot more flexibility than discrete
#it allows you to pass through multiple values
#we are using a single value array, but we can use a n-dimensional tensor/space and could be used to hold dataframes, images and audio



        self.state = 38 + random.randint(-3,3)
# Set start temp | set state into the observation space       
    

        self.shower_length = 60
# Set shower length | the time dimension for 60 seconds
    
#step function is running for each step in our environment
#add action as another parameter for the step function
#action is the action_space from __init_ function 0,1,2 or down, stay, up
#we apply the action to the state which is set to 38 in the observation space of 0 to 100
    def step(self, action):
        
        self.state += action -1     
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        
        self.shower_length -= 1 
        # Reduce shower length by 1 second
        
     # Calculate reward | should be a reward function
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
    
    # Apply temperature noise | if you drop it off, your model is most likely faster
        self.state += random.randint(-1,1)
        # Set placeholder for info, this is something opengym requires
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    
    def render(self):
        #implement viz optional
        pass
    
    
    
    def rest(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state 


In [6]:
# finished version
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [7]:
#For testing it out, create a new environment of shower instance
env = ShowerEnv()

C:\Users\nadam\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [8]:
#testing observation space
env.observation_space.sample()

array([96.34083], dtype=float32)

In [9]:
#testing action_space
env.action_space.sample()

1

In [10]:
# run batches
episodes = 10 #doing 10 showers
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-58
Episode:2 Score:10
Episode:3 Score:-60
Episode:4 Score:38
Episode:5 Score:-32
Episode:6 Score:-4
Episode:7 Score:-2
Episode:8 Score:-40
Episode:9 Score:-60
Episode:10 Score:-28


# 2. Create a Deep Learning Model with Keras

In [2]:

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam # - Works





In [11]:
states = env.observation_space.shape
actions = env.action_space.n

In [12]:
actions

3

In [13]:
states

(1,)

In [14]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
del model

NameError: name 'model' is not defined

In [16]:
model = build_model(states, actions)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [18]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [21]:
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 11:23 - reward: 1.0000

C:\Users\nadam\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\nadam\AppData\Local\Programs\Python\Python310\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 70s 7ms/step - reward: -0.3436
166 episodes - episode_reward: -20.554 [-60.000, 60.000] - loss: 0.589 - mae: 3.352 - mean_q: -1.619

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: -0.4038
167 episodes - episode_reward: -24.383 [-60.000, 52.000] - loss: 1.122 - mae: 6.559 - mean_q: -9.154

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: -0.4026
167 episodes - episode_reward: -24.048 [-60.000, 32.000] - loss: 1.478 - mae: 8.012 - mean_q: -11.343

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -0.4398
166 episodes - episode_reward: -26.325 [-60.000, 44.000] - loss: 1.550 - mae: 8.190 - mean_q: -11.603

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: -0.3630
done, took 366.837 seconds


In [22]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: 58.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 58.000, steps: 60
Episode 21: reward: 58.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.00

In [31]:
test = dqn.test(env, nb_episodes=15, visualize=False)
#test = dqn.test(env, nb_episodes=15, visualize=True)
#scores = dqn.test(env, nb_episodes=100, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 58.000, steps: 60
Episode 14: reward: 58.000, steps: 60
Episode 15: reward: -60.000, steps: 60


# 4. Reloading Agent from Memory

In [24]:
#dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
#del model
#del dqn
#del env

In [ ]:
#env = gym.make('CartPole-v0')
#actions = env.action_space.n
#states = env.observation_space.shape[0]
#model = build_model(states, actions)
#dqn = build_agent(model, actions)
#dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
#dqn.load_weights('dqn_weights.h5f')

In [34]:
Test = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
